<a href="https://colab.research.google.com/github/jmbost20/YouTransfer/blob/main/Get_youtube_transcripts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Begin by installing custom youtube api implementation wrapper (youtube_transcript_api) and custom web scraper (scrapetube)

These will allow the bypass of expensive Youtube API calls

So far the following issues have been identified: 
1. Not all video ids are collected (in the ElectroBOOM example case 10 of 208 videos are not collected) 
2. Some transcripts fail to be collected (in the ElectroBOOM example case 10 of 198 videos are not collected)
3. Sentences are broken between transcript tokens and are without delimiters
4. Transcripts (not manually created transcripts) are dependent on Google's machine translation and thus some words are transcribed incorrectly
5. The Historia Civilis df has incorrect channel id due to bug (need to fix with correct channel id at some point) (bug is already fixed)

Investigate
1. Videos not collected- Protected/Hidden/Private or Deleted Videos? 
3. Check if Transcripts are autogenerated


Current Transcripts:
Educational: [ElectroBOOM, Historia Civilis, Kings and Generals, PBS Eons, Moth Light Media, 3Blue1Brown]
Math/History/Biology

Important URLS: 
1. https://stackoverflow.com/questions/15512239/python-get-all-youtube-video-urls-of-a-channel -ScrapeTube Reference
2. https://pypi.org/project/youtube-transcript-api/#description - transcript api source
3. https://commentpicker.com/youtube-channel-id.php - Easily grab channel id
4. https://pytube.io/en/latest/user/install.html - Easily grab audio from video id


Go to channel on youtube, get url, put it into #3, and receive basic summary information about that channel. We want the channel ID. Once we have that...

In [ ]:
pip install youtube_transcript_api


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install scrapetube

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import scrapetube
from youtube_transcript_api import YouTubeTranscriptApi
import pandas as pd


In [ ]:
#simple function for aggregating text but still maintains transcript token format 
# (returns list of text snippets)
def get_full_transcript(raw_transcript):
  text_collection = []
  for dictionary in raw_transcript:
    ###write code to list all dictionary keys
    print(dictionary['text'])
    text_collection.append(dictionary['text'] + '.')
  return text_collection

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


First Use ScrapeTube to get youtube video list without API call. Note you must use the following website to easily get a youtuber's channel ID: 
https://commentpicker.com/youtube-channel-id.php


Note: potentially consider using each dict from transcript call as a token of speech instead of a whole sentence


In [ ]:
%%time
channel_id = 'UCnxmUrGMtpQT844Yd_l7Zyg'
Channel_Name =  'Dr Dray'
file_path = f'/content/drive/MyDrive/Style-Transfer/Youtuber-Transcripts/{Channel_Name}.csv'   #Note this may need to change depending on the machine, ideally we will have a shared folder that is accessible to all using same path
videos = scrapetube.get_channel(channel_id)

video_id_list = []
for video in videos:
    video_id_list.append(video['videoId'])


length_list = range(len(video_id_list)) #need range to make a list when initializing df
current_channel_videos_df = pd.DataFrame({'Channel_Name':[Channel_Name for i in length_list], 'Channel_ID':[channel_id for i in length_list],'Raw_Transcript':[0 for i in length_list],'Full_Transcript':[0 for i in length_list]})
current_channel_videos_df.index = video_id_list  #need for simple indexing with .at
current_channel_videos_df['Raw_Transcript'] = current_channel_videos_df['Raw_Transcript'].astype('object')   #need this to set list value with .at
current_channel_videos_df['Full_Transcript'] = current_channel_videos_df['Full_Transcript'].astype('object') #need this to set list value with .at


CPU times: user 1.85 s, sys: 101 ms, total: 1.95 s
Wall time: 1min 32s


In [ ]:
current_channel_videos_df

,Channel_Name,Channel_ID,Raw_Transcript,Full_Transcript
ePWhRRBB1mw,Dr Dray,UCnxmUrGMtpQT844Yd_l7Zyg,0,0
1dLCyXHTHKQ,Dr Dray,UCnxmUrGMtpQT844Yd_l7Zyg,0,0
aALYla5Uqt4,Dr Dray,UCnxmUrGMtpQT844Yd_l7Zyg,0,0
kLiyfvND-AI,Dr Dray,UCnxmUrGMtpQT844Yd_l7Zyg,0,0
qRrgOgl-GlI,Dr Dray,UCnxmUrGMtpQT844Yd_l7Zyg,0,0
...,...,...,...,...
AzU-pLHtfhE,Dr Dray,UCnxmUrGMtpQT844Yd_l7Zyg,0,0
5j2OUIqAc-g,Dr Dray,UCnxmUrGMtpQT844Yd_l7Zyg,0,0
Y1dNSM79JcM,Dr Dray,UCnxmUrGMtpQT844Yd_l7Zyg,0,0
8okZvc1BMLc,Dr Dray,UCnxmUrGMtpQT844Yd_l7Zyg,0,0


In [ ]:
%%time
for vid_id in video_id_list:
  try:
    raw_transcript = YouTubeTranscriptApi.get_transcript(vid_id)
    current_channel_videos_df.at[vid_id,'Raw_Transcript'] = raw_transcript
  except: 
    raw_transcript = False
    current_channel_videos_df.at[vid_id,'Raw_Transcript'] = raw_transcript
  if raw_transcript == False:
    current_channel_videos_df.at[vid_id,'Full_Transcript'] = raw_transcript
  else: 
    current_channel_videos_df.at[vid_id,'Full_Transcript'] = get_full_transcript(raw_transcript)
current_channel_videos_df.to_csv(file_path)

Streaming output truncated to the last 5000 lines.
you've got any good Halloween ideas and
no don't just say doctor and scrubs
because that's lame
okay what's up YouTube on my way back
from work had a really great clinic
today a lot of good cases interesting
patients and and it's Friday so I
decided to go to the store
two pickups essentials
spinach namely
and I'm thinking of making a pumpkin
pancakes this weekend or yeah so I was
thinking as far as a Halloween costume
that would be appropriate for work that
maybe I could get cat or animal ears
found and I'm doing surgery on that
Monday I thought it would be cute if I
got a surgical mask and I decorated it
with like the little animal face what's
up YouTube I am back them I went to the
store and got some things and then I
went for my run and then I got some
canned pumpkin that got me thinking
about today's fun fact and I whipped out
my Costco cinnamon one of the things I
wanted to share with you guys as you're
baking this fall season cin

In [ ]:
current_channel_videos_df.index

Index(['ePWhRRBB1mw', '1dLCyXHTHKQ', 'aALYla5Uqt4', 'kLiyfvND-AI',
       'qRrgOgl-GlI', '2LEKd0Z5Uho', 'mFXin5bSXbs', 'zM0qUIWgPnw',
       'xmN1oIVJlLM', 'f9d9Fbj9_Bo',
       ...
       '1gyUGU6k4so', 'gedXuJD7us4', 'WuHqGzaZNbQ', 'L4DLkk45P9k',
       'dlCTbcqzO9I', 'AzU-pLHtfhE', '5j2OUIqAc-g', 'Y1dNSM79JcM',
       '8okZvc1BMLc', 'C0n6gT-V-QE'],
      dtype='object', length=2428)

In [ ]:
len(current_channel_videos_df[current_channel_videos_df['Raw_Transcript']==False]) #check how many transcripts are missing

80

In [ ]:
for vid_id in video_id_list:
  current_channel_videos_df.at[vid_id,'Test_col'] = get_full_transcript(raw_transcript)


morning YouTube it is Saturday September
17th and I
okay good audience video and I wash my
face it's water now a dress shirt so
right now I'm making my coffee yeah I
don't know to face time with my mom I
worry about what to do Saturday morning
I saw my mom my mom lives in Colorado I
don't get to see her very often so oh I
tried FaceTime with her at least once a
week which makes up for the fact that I
don't get to see her all the time
anyways so that's on the docket first
and then I made an appointment last
night to test drive a Mazda 3 so the guy
is supposed to email me this morning
they open at 9:00
you know where the day takes us
great so I'm probably gonna get some
coffee at fellini's before I go because
that one little Starbucks instant thing
is just not gonna not going to get me
through what I need to it'll be off
here we go all right so I got my coffee
and I just love that place they're so
nice in there so the other thing that I
find frustrating in my apartment
building is so I'v

ValueError: ignored

In [ ]:
current_channel_videos_df.loc['9dawNcG_ue8']['Full_Transcript']